In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
true = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
fake = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')

In [ ]:
true.head(10)

In [ ]:
fake.head(10)

In [ ]:
true['label'] = 'fake'
fake['label'] = 'true'

data = pd.concat([true, fake], axis = 0)
data.head(10)

In [ ]:
data = data[['text', 'label']]
data.head(10)

In [ ]:
print('length of dataset is:' ,len(data))

In [ ]:
data['text'] = data['text'].str.lower()

In [ ]:
data['text'].head(10)

In [ ]:
data.columns.values.tolist()

In [ ]:
data['label'].unique().tolist()

In [ ]:
data.dtypes

In [ ]:
data[data.isnull().any(axis = 1)]

In [ ]:
import numpy as np
np.sum(data.isnull().any(axis = 1))

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style = 'whitegrid')
carrier_count = data['label'].value_counts()
sns.barplot(carrier_count.index, carrier_count.values, alpha = 0.9)
plt.title('Distribution of label')
plt.xlabel('Carrier')
plt.ylabel('Number of Occurances')
plt.legend()
plt.show()

In [ ]:
data['label'].value_counts().plot(kind = 'pie', autopct = '%1.1f%%', figsize = (8,8)).legend()

In [ ]:
data['label'].value_counts().plot(kind = 'pie', autopct = '%1.1f%%', figsize = (8,8)).legend()

In [ ]:
data.loc[data['text'].str.contains('america'), 'text'] = 'american'
data.loc[data['text'].str.contains('americans'), 'text'] = 'american'
data.loc[data['text'].str.contains('youore'), 'text'] = 'you are'
data.loc[data['text'].str.contains('youre'), 'text'] = 'you are'
data.loc[data['text'].str.contains('years'), 'text'] = 'year'
data.loc[data['text'].str.contains('break'), 'text'] = 'breakingnews'
data.loc[data['text'].str.contains('breaking'), 'text'] = 'breakingnews'
data.loc[data['text'].str.contains('caused'), 'text'] = 'cause'
data.loc[data['text'].str.contains('causes'), 'text'] = 'cause'
data.loc[data['text'].str.contains('county'), 'text'] = 'country'

In [ ]:
data['text'].str.find('american').value_counts()


In [ ]:
for letter in '1234567890.(/':
    data['text'] = data['text'].str.replace(letter, ' ')

In [ ]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations + english_punctuations

In [ ]:
def remove_punctuations(text):
    translator = str.maketrans('','', punctuations_list)
    return text.translate(translator)

In [ ]:
import re
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def processPost(text): 

    text = re.sub('@[^\s]+', ' ', text)
    

    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)

    text = re.sub(r'#([^\s]+)', r'\1', text)

    text= remove_punctuations(text)
    text=remove_repeating_char(text)
    
    return text

In [ ]:
data['text'] = data['text'].apply(lambda x: processPost(x))

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
data['text'] = data['text'].apply(tokenizer.tokenize)

In [ ]:
data['text'].head(10)

In [ ]:
from nltk.corpus import stopwords
stopwords_list = set(stopwords.words("english"))

In [ ]:
stopwords_list

In [ ]:
data['text'] = data['text'].apply(lambda x: [item for item in x if item not in stopwords_list])

In [ ]:
data['text'].head(10)

In [ ]:
all_words = [word for tokens in data['text'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['text']]

vocab = sorted(list(set(all_words)))

print(len(all_words), len(vocab))
print( max(sentence_lengths))

In [ ]:
from collections import Counter
counter = Counter(all_words)
counter.most_common(25)

In [ ]:
counted_words = counter.most_common(25)

word = []
counts = []
for letter, count in counted_words:
    word.append(letter)
    counts.append(count)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(word, counts, alpha = 0.9)
plt.xticks(rotation = 90)
plt.title("Frequency of most common words")
plt.xlabel("Most common words")
plt.ylabel("count of words")

In [ ]:
data1 = data[data['label'] == 'true']
neg = data1['text']
neg_string = []
for t in neg:
    neg_string.append(t)
neg_str = pd.Series(neg_string).map(str)
neg_str = str(neg_str)

In [ ]:
from wordcloud import WordCloud
wordcloud = WordCloud(width = 1600, height = 800, max_font_size = 200).generate(neg_str)
plt.figure(figsize = (12,10))
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
data2 = data[data['label'] == 'fake']
neg = data2['text']
neg_str = []
for t in neg:
    neg_str.append(t)
neg_str = pd.Series(neg_str).map(str)
neg_str = str(neg_str)

In [ ]:
wordcloud = WordCloud(width = 1600, height = 800, max_font_size = 200).generate(neg_str)
plt.figure(figsize = (12,10))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

word_vectorizer = TfidfVectorizer(
    sublinear_tf = True,
    strip_accents = 'unicode',
    analyzer = 'word',
    token_pattern = r'\w{1,}',
    stop_words = 'english',
    ngram_range = (1,1),
    max_features = 1000)

In [ ]:
unigramdataGet = word_vectorizer.fit_transform(data['text'].astype('str'))
unigramdataGet = unigramdataGet.toarray()
vocab = word_vectorizer.get_feature_names()
unigramdata = pd.DataFrame(np.round(unigramdataGet,1), columns = vocab)
unigramdata[unigramdata>0] = 1
unigramdata.head()


In [ ]:
vocab[0:100]

In [ ]:
len(vocab)

In [ ]:
len(unigramdata)

In [ ]:
data.reset_index(drop = True, inplace = True)
data = data.drop(columns = ['text'])

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

selector = SelectKBest(chi2, k = 700)
selector.fit(unigramdata, data['label'])

unigramdata = selector.transform(unigramdata)
unigramdata.shape

In [ ]:
X = unigramdata
y = data['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 100)

In [ ]:
from sklearn.svm import LinearSVC

svc = LinearSVC(C = 1, max_iter = 500)
svc = svc.fit(X_train, y_train)
svc

In [ ]:
y_pred1 = svc.predict(X_test)
dm = svc.score(X_test, y_test)
print('Accuracy Score - {:.2f}'.format(dm))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
CR = classification_report(y_test, y_pred1)
print(CR)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
CR = confusion_matrix(y_test, y_pred1)
print("Confusion Matrix:-")
print(CR)

In [ ]:
plot_confusion_matrix(svc,X_test, y_test)

plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [ ]:
raf = RandomForestClassifier(min_samples_leaf = 20, min_samples_split = 20, random_state = 100)
raf = raf.fit(X_train, y_train)

In [ ]:
y_pred1 = raf.predict(X_test)
rf = raf.score(X_test, y_test)
print("Accuracy score is {:0.2f}".format(rf))

In [ ]:
CR = classification_report(y_test, y_pred1)
print("Classification Report is :-")
print(CR)

In [ ]:
CR = confusion_matrix(y_test, y_pred1)
print(CR)

In [ ]:
plot_confusion_matrix(raf, X_test, y_test)
plt.show()

In [ ]:
from sklearn.ensemble import VotingClassifier

Ens = VotingClassifier(estimators = [('SVM', svc),
                                    ('raf', raf)], voting = 'hard')
Ens = Ens.fit(X_train, y_train)
Ens

In [ ]:
y_pred1 = Ens.predict(X_test)
acc = Ens.score(X_test, y_test)
acc